### Lab 3 project

#### 1st part: Download training and test files to local folder

In [3]:
import requests,openpyxl,sqlite3

url = "http://kevincrook.com/utd/market_basket_training.txt"

r = requests.get(url)
type(r.content)
f = open("market_basket_training.txt","wb")
f.write(r.content)


url = "http://kevincrook.com/utd/market_basket_test.txt"

r = requests.get(url)
type(r.content)
f = open("market_basket_test","wb")
f.write(r.content)

f.close()

#### 2nd part: Import data into sqllite3 table and aggregate raw data to generate stat results

In [1]:
import csv, sqlite3

sqlite_file = 'staging.db'    # name of the sqlite database file

con = sqlite3.connect(sqlite_file)
c = con.cursor()

#Create Tbls I need
tbl_nm = 'tblTraining'
c.execute('CREATE TABLE {tn} (T_NUM TEXT, Activity TEXT) '.format(tn=tbl_nm))

tbl_nm = 'tblTraning_Ref_Count'
c.execute('CREATE TABLE {tn} (DISTINCT_ACTIVITY TEXT, COUNT INTEGER) '.format(tn=tbl_nm))

con.commit()

#Import data section
tbl_nm = 'tblTraining'
with open("market_basket_training.txt", mode='r', encoding="cp1252") as f:
    reader = csv.reader(f, delimiter="\n")
    c.executemany('INSERT INTO {tn} (Activity) VALUES (?)'.format(tn=tbl_nm), reader)

c.execute('UPDATE {tn} SET Activity = replace(Activity,",", "")'.format(tn=tbl_nm))
c.execute('UPDATE {tn} SET Activity = substr(Activity, instr(Activity, "P")) '.format(tn=tbl_nm))
con.commit()

#Create stat in tblTraning_Ref_Count
c.execute('INSERT INTO tblTraning_Ref_Count (DISTINCT_ACTIVITY, COUNT) SELECT Activity, COUNT(*) FROM {tn} GROUP BY Activity ORDER BY Activity'.format(tn = tbl_nm))
con.commit()


#### 3rd part: Load test data into sqllite3 table

In [8]:
import sqlite3
sqlite_file = 'staging.db'    # name of the sqlite database file
con = sqlite3.connect(sqlite_file)
c = con.cursor()

tbl_nm = 'tblTraining' 

c.execute('DELETE FROM {tn}'.format(tn=tbl_nm))
con.commit()

with open("market_basket_test.txt", mode='r', encoding="cp1252") as f:
    reader = csv.reader(f, delimiter="\n")
    c.executemany('INSERT INTO {tn} (Activity, T_NUM) VALUES (?, "")'.format(tn=tbl_nm), reader)

c.execute('UPDATE {tn} SET T_NUM = substr(Activity, 0,4) '.format(tn=tbl_nm))   
c.execute('UPDATE {tn} SET Activity = substr(Activity, instr(Activity, "P")) '.format(tn=tbl_nm))

con.commit()


#Check data in tables----PLEASE IGNORE THIS PART
#c.execute('SELECT Activity, T_NUM FROM tblTraining')
#all_rows = c.fetchall()
#print(all_rows)

#### 4th part: Generate recommend product from test

In [14]:
import sqlite3

def replaceNull(x):
  if x is None:
    return ''
  else:
    return x

#Get it ready
recommend_file = open("market_basket_recommendations.txt", "w")

#This comes from requirement, "The company only has 10 products"
product_list = ['P01','P02','P03','P04','P05','P06','P07','P08','P09','P10']
product_missing = []
    
sqlite_file = 'staging.db'    # name of the sqlite database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

#Generate product_missing list
for i, s in enumerate(product_list):
    c.execute('SELECT CASE WHEN SUM(COUNT) > 1 THEN "GOOD" ELSE "BAD" END FROM {tb} WHERE instr(DISTINCT_ACTIVITY, "{value}") > 0 '.\
          format(tb='tblTraning_Ref_Count', value = s))
    count_row = c.fetchall()

    for r, row in enumerate(count_row):
        if str(row[0]) == "BAD":
            product_missing.append(s)

#print(product_missing)

tbl_nm = 'tblTraining'
c.execute('SELECT Activity, T_NUM FROM {tb} '.format(tb=tbl_nm))
all_rows = c.fetchall()


my_test = ""
my_test1 = "AND instr(DISTINCT_ACTIVITY, \""
my_test2 = "\") > 0 "


for r, row in enumerate(all_rows):
    
    my_test = ""
    remove_product = ""
    P_CNT = row[0].count('P') + 1 # I need to get P02P04 total P count, then add one to include one recommend product
    
    for x in row[0].split(','):
        if x.strip() not in product_missing:
            my_test = my_test + my_test1 + x.strip() + my_test2
        else:
            P_CNT = P_CNT -1
            remove_product = x.strip()

    stmt = "SELECT MAX(COUNT), DISTINCT_ACTIVITY FROM tblTraning_Ref_Count WHERE length(DISTINCT_ACTIVITY) - length(replace(DISTINCT_ACTIVITY, \'P\', \'\')) = "+ str(P_CNT) +" " + my_test
    c.execute(stmt)
    
    result_rows = c.fetchall()
    
    recommend_product = ""
    #Do clean up and only leave recommended product code
    for y in replaceNull(result_rows[0][1]).split('P'):
        if y != '':
            #print("P"+y)
            if row[0].count("P"+y) == 0:
                recommend_product = "P"+y
                

    #if (remove_product != ""):
        #print("We remove product:",remove_product)
    recommend_file.write(row[1]+","+recommend_product+"\n")
    

conn.commit()
recommend_file.close()

print("IF YOU SAW THIS, IT MEANS WE'RE DONE, GO TO SAME FOLDER TO CHECK OUT FILE!HAPPY HOLIDAYS!")

IF YOU SAW THIS, IT MEANS WE'RE DONE, GO TO SAME FOLDER TO CHECK OUT FILE!HAPPY HOLIDAYS!
